# Loading Battery Cycler Data to PyBOP

This notebook provides example on how to load battery cycler data without any pre-processing and save it so that this data can be used to run simulations using PyBOP.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. In this example, we use Battery Data Toolkit to load cycler data. More information can be found here: https://github.com/ROVI-org/battery-data-toolkit.

In [6]:
%pip install --upgrade pip ipywidgets openpyxl pandas -q
%pip install pybop -q
#%pip uninstall bpx

%pip install battery-data-toolkit -q

import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
import pybamm
from batdata.data import BatteryDataset

import pybop

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
emmontopy 0.7.1 requires pandas<2.2,>=1.2, but you have pandas 2.2.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In this example, we use battery cycler data from Logan Ward et al. Please refer to DOI: 10.18126/fdxq-7yul for more details. The data is in H5 format. An H5 is one of the Hierarchical Data Formats (HDF) used to store large amount of data in the form of multidimensional arrays.

In [7]:
data_path = r"../data/cycler_data/batch_B6D_cell_6.h5"
dataset = BatteryDataset.from_batdata_hdf(data_path)

To see the metadata which contains cell details, batch information etc., we can use the following commnad.

In [8]:
dataset.metadata.model_dump(exclude_defaults=True)

{'name': 'CAMP_batch_B6D_cell_6',
 'battery': {'layer_count': 13,
  'anode': {'name': 'C',
   'supplier': 'Commercial',
   'product': 'Graphite',
   'thickness': 100.0,
   'loading': 7.3,
   'porosity': 33.0},
  'cathode': {'name': 'NMC111',
   'supplier': 'Commercial',
   'thickness': 67.0,
   'area': 169.2,
   'loading': 15.21,
   'porosity': 29.0},
  'electrolyte': {'name': 'Gen 2'},
  'nominal_capacity': 0.3},
 'dataset_name': 'camp_2023',
 'authors': [('Logan', 'Ward'),
  ('Joseph', 'Kubal'),
  ('Susan J.', 'Babinec'),
  ('Wenquan', 'Lu'),
  ('Allison', 'Dunlop'),
  ('Steve', 'Trask'),
  ('Andrew', 'Jansen'),
  ('Noah H.', 'Paulson')],
 'associated_ids': [Url('https://doi.org/10.1016/j.jpowsour.2022.231127')]}

The following code block gives a summary of the cycle statistics.

In [9]:
dataset.cycle_stats

,cycle_number,energy_charge,capacity_charge,energy_discharge,capacity_discharge,cycle_start,cycle_duration
0,0,0.000000,0.000000,0.000001,4.253452e-07,0.000,120.060
1,1,1.091499,0.299519,1.064703,2.829004e-01,120.060,42211.818
2,2,1.090546,0.299251,1.123087,3.011855e-01,42331.878,43509.642
3,3,1.089339,0.298922,1.120073,3.003821e-01,85841.520,43427.832
4,4,1.040652,0.287874,1.120398,2.975029e-01,129269.352,17396.970
...,...,...,...,...,...,...,...
214,214,0.278841,0.088494,0.344245,8.650017e-02,3445507.056,5722.188
215,215,0.288119,0.091488,0.355268,8.930149e-02,3451229.244,5900.148
216,216,0.226854,0.076921,0.364280,9.159989e-02,3457129.392,7978.644
217,217,0.292792,0.104235,0.322057,7.880164e-02,3465108.036,43793.880


Now from the raw data, we can extract necessary parameters that we will use later in PyBOP. In this case, we are interested to save time, current, and voltage data.

In [10]:
# loading the raw data
df = dataset.raw_data

# peeping in the raw data to know the column names
print(df)

# extracting time, current, and voltage from the data
time = df["test_time"]
current = df["current"]
voltage = df["voltage"]

       cycle_number  file_number    test_time    state  current   voltage  \
0                 0            0        0.000     hold     -0.0  3.425956   
1                 0            0       10.002     hold     -0.0  3.426413   
2                 0            0       19.998     hold     -0.0  3.426261   
3                 0            0       30.000     hold     -0.0  3.426566   
4                 0            0       40.002     hold     -0.0  3.426566   
...             ...          ...          ...      ...      ...       ...   
33011           218            4  3887383.206     hold     -0.0  2.689250   
33012           218            4  3888583.206     hold     -0.0  2.689555   
33013           218            4  3889783.206     hold     -0.0  2.689555   
33014           218            4  3890850.546     hold     -0.0  2.689860   
33015           218            4  3890850.546  unknown     -0.0  2.689708   

       step_index         method  substep_index  cycle_capacity  cycle_ener

Let's create an interactive plot to see how current varies with time. Only first 10000 data points are shown to reduce the file size. Users can customize this as per their need.

In [11]:
# To see current vs time profile.
# Create traces for current and voltage

datapoints = 10000

trace1 = go.Scatter(
    x=time[0:datapoints],
    y=current[0:datapoints],
    mode="lines+markers",
    name="Current",
    line=dict(color="blue"),
)

######################################################
# Plot current
data = [trace1]

# Define the layout
layout = go.Layout(
    title="Current vs Time",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Current (A)"),
    legend=dict(x=0, y=1),  # Legend position
)
######################################################

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
pyo.iplot(fig)

Now we create voltage vs time interactive plot for the same number of data points.

In [12]:
# To see voltage vs time profile.
# Create traces for current and voltage

trace2 = go.Scatter(
    x=time[0:datapoints],
    y=voltage[0:datapoints],
    mode="lines+markers",
    name="Voltage",
    line=dict(color="red"),
)

######################################################
# Plot voltage
data = [trace2]

# Define the layout
layout = go.Layout(
    title="Voltage vs Time",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Voltage (V)"),
    legend=dict(x=0, y=1),  # Legend position
)
######################################################

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
pyo.iplot(fig)  # Use pyo.plot(fig) to save as an HTML file

Now we will save time, current, and voltage data in the form of a compressed numpy array.

In [13]:
# save time, current, and voltage to a numpy file
np.savez_compressed(
    "../data/cycler_data/time_current_voltage.npz",
    time=time,
    current=current,
    voltage=voltage,
)

Import the saved time, current, and voltage data.

In [14]:
# Load the SOC and OCV data from the previously saved numpy file
data = np.load("../data/cycler_data/time_current_voltage.npz")
time = data["time"]
current = data["current"]
voltage = data["voltage"]

From the data, we now choose an HPPC pulse. From the current vs time interactive graph plotted earlier, we can identify HPPC pulses. We then use some logical argumemt to find the corresponding index of the time data column.

In [15]:
# identify start and end time of an HPPC pulse from the interactive plot
start_time = 238664.2
end_time = 239209.4  # 238913
start_index = np.argmin(np.abs(time - start_time))
end_index = np.argmin(np.abs(time - end_time))

Let's plot the current and voltage data against time for the chosen HPPC pulses.

In [16]:
# Visualizing current and voltage profiles

# Create traces for current and voltage
trace1 = go.Scatter(
    x=time[start_index:end_index],
    y=current[start_index:end_index],
    mode="lines+markers",
    name="Current",
    line=dict(color="blue"),
    yaxis="y1",
)

trace2 = go.Scatter(
    x=time[start_index:end_index],
    y=voltage[start_index:end_index],
    mode="lines+markers",
    name="Voltage",
    line=dict(color="red"),
    yaxis="y2",
)

data = [trace1, trace2]

# Define the layout
layout = go.Layout(
    title="Current and Voltage vs Time",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(
        title="Current (A)", titlefont=dict(color="blue"), tickfont=dict(color="blue")
    ),
    yaxis2=dict(
        title="Voltage (V)",
        titlefont=dict(color="red"),
        tickfont=dict(color="red"),
        overlaying="y",
        side="right",
    ),
    legend=dict(x=0.8, y=0.8),  # Legend position
)
######################################################

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
pyo.iplot(fig)

In this example, we use the default parameter value for the "Open-circuit voltage [V] as provided by the original PyBaMM class. To update this, provide a function definition that matches this [function](https://github.com/pybamm-team/PyBaMM/blob/1943aa5ab2895b5378220595923dbae3d66b13c9/pybamm/input/parameters/ecm/example_set.py#L17).

In [17]:
# Load the parameters
parameter_set = pybop.ParameterSet.pybamm("ECM_Example")

Update the battery parameters.

In [18]:
# Update the ECM input parameter as required for the present case
parameter_set.update(
    {
        "Cell capacity [A.h]": 0.3,
        "Nominal cell capacity [A.h]": 0.3,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 2.5,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 3e-3,
        "C1 [F]": 5e2,
        "Open-circuit voltage [V]": pybop.empirical.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ],
    }
)
# Optional arguments - only needed for two RC pairs
parameter_set.update(
    {
        "R2 [Ohm]": 0.002,
        "C2 [F]": 3e4,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

### Identifying the Parameters

Now that the initial parameter set is constructed, we can start the PyBOP fitting process. First, we define the model class with two RC elements.

In [19]:
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set,
    options={"number of rc elements": 2},
    solver=pybamm.CasadiSolver(mode="safe", dt_max=10),
)

### Create PyBOP dataset.

In [20]:
new_time = time[start_index:end_index]
new_time = new_time - new_time[0]
new_current = current[start_index:end_index]
new_voltage = voltage[start_index:end_index]

dataset = pybop.Dataset(
    {
        "Time [s]": new_time,
        "Current function [A]": new_current,
        "Voltage [V]": new_voltage,
    }
)
r0_guess = 0.005

In [21]:
parameters = pybop.Parameters(
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.5],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.5],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.5],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[100, 10000],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[100, 10000],
    ),
)

The `FittingProblem` class provides us with a single class that holds all of the objects we need to evaluate our selected `SumSquaredError` cost function. 

In [22]:
model.build(initial_state={"Initial open-circuit voltage [V]": new_voltage[0]})
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the XNES method as it provides global optimisation capability.

In [23]:
optim = pybop.XNES(
    cost,
    sigma0=[1e-3, 1e-3, 1e-3, 20, 20],
    max_unchanged_iterations=50,
    max_iterations=250,
)
x, final_cost = optim.run()
print("Initial parameters:", optim.x0)
print("Estimated parameters:", x)

/home/dibyendu/Oxford_Work/PyBOP/.venv/lib/python3.10/site-packages/pybamm/solvers/base_solver.py:762: SolverWarning:

Explicit interpolation times not implemented for CasADi solver with 'safe' mode



Initial parameters: [4.76293360e-03 5.15973471e-03 5.02345920e-03 4.81204363e+02
 2.28029289e+03]
Estimated parameters: [2.17490811e-01 3.59643257e-02 1.70578507e-01 1.32665976e+02
 4.16760058e+02]


### Plotting and Visualisation

PyBOP provides various plotting utilities to visualize the results of the optimisation.

In [24]:
pybop.quick_plot(problem, problem_inputs=x, title="Optimised Comparison");

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [25]:
pybop.plot_convergence(optim)
pybop.plot_parameters(optim);